In [59]:
import numpy as np
import pandas as pd
import nltk
import re

In [60]:
df = pd.read_csv("data2.csv",encoding='mac_roman')

In [83]:
required_title="Machine Learning"
idx=df['article_title'].str.find(required_title)
req_idx=[]
c=0
for i in idx:
    if(i!=-1):
        req_idx.append(c)
    c=c+1
print(req_idx)

[0]


In [96]:
from nltk.tokenize import sent_tokenize
sentences = []
for i in range(len(df)):
    if(df['article_title'][i].find(required_title)!=-1):
        sentences.append(sent_tokenize(df['article_text'][i]))

sentences = [y for x in sentences for y in x]

In [97]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]

In [98]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [99]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [106]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [107]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [108]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [109]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [110]:
from sklearn.metrics.pairwise import cosine_similarity

In [111]:
sentences

['Machine learning (ML) is the study of computer algorithms that improve automatically through experience.',
 '[1] It is seen as a subset of artificial intelligence.',
 'Machine learning algorithms build a mathematical model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so.',
 '[2] Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks.',
 'Machine learning is closely related to computational statistics, which focuses on making predictions using computers.',
 'The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.',
 'Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learning.',
 '[4][5] In its application across business proble

In [112]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [113]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [124]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=False)

In [126]:
for i in range(3):
  print(ranked_sentences[i][1])

[1] It is seen as a subset of artificial intelligence.
Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learning.
The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.
